In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from functools import reduce
from scipy.stats import linregress
from scipy import fftpack
from scipy.signal import detrend
from tqdm import tqdm
from scipy.optimize import root_scalar
from tqdm.notebook import tqdm_notebook
from scipy.optimize import curve_fit
from tsmoothie.smoother import LowessSmoother
import matplotlib
from statsmodels.tsa.stattools import acf, pacf
import mpl_interactions.ipyplot as iplt
from sklearn.metrics import r2_score
from scipy.stats import linregress
from scipy.signal import detrend, welch, spectrogram
import plotly.express as px 
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.graph_objects as go

In [2]:
def psi(m, b):
    return (1/10) * b**(-1/10) * m**(-0.9)

def cumulativeM(m, b):
    return b**(-0.1) * m**(0.1)

def inverseCumM(u, b):
    return b * u**(10)

In [3]:
def pdfLogSpace(x, xb, gamma):
    return 2**(- gamma) * gamma * np.exp(gamma*(- x + xb + np.exp(xb) - 2*np.exp(x))+ np.log(1 + 2 * np.exp(x))) 


def cdfLogSpace(x, xb, gamma):
    return 1  - 2**(- gamma) * np.exp(gamma * (-2 * np.exp(x) + np.exp(xb) - x + xb))


def  drawCDF(unif, gamma, xb):
    #if np.sign(cdfLogSpace(-100, gamma = gamma, xb = xb) - unif) * np.sign(cdfLogSpace(50, gamma = gamma, xb = xb) - unif) == 1:
        #print(gamma, xb, unif, cdfLogSpace(-200, gamma = gamma, xb = xb) - unif, cdfLogSpace(50, gamma = gamma, xb = xb) - unif)
    return root_scalar(lambda t: cdfLogSpace(t, gamma = gamma, xb = xb) - unif, bracket=[-400, 50], method='brentq').root

In [4]:
criticalPoint = 1 / np.log(2)
gammaValues = - np.abs(np.logspace(np.log10(np.abs(1 - criticalPoint)), np.log10(1e-3), 100))  + criticalPoint
gamma_gammaC = np.abs(np.logspace(np.log10(np.abs(1 - criticalPoint)), np.log10(1e-3), 100))

In [5]:
def sizeAtBirth(gamma, xb0, seriesLength = 5000):
    uniformDraws = np.random.uniform(size = seriesLength)

    logSizes = np.zeros(seriesLength)
    logSizes[0] = xb0
    for i in range(1, seriesLength):
        logSizes[i] = drawCDF(unif = uniformDraws[i], gamma = gamma, xb = logSizes[i-1])

    return logSizes

In [6]:
b = 10
seriesLength = 20400
simulate = False
startingPointsUniform = np.random.uniform(0, 1, len(gammaValues))
b = 10
def inverseCumM(u, b):
    return b * u**(10)
startingPoints = inverseCumM(startingPointsUniform, b)
if simulate:
    allSizes = np.zeros((len(gammaValues), seriesLength))
    for i, gamma in enumerate(tqdm_notebook(gammaValues)):
        sizeBirth = sizeAtBirth(gamma = gamma, xb0 = startingPointsUniform[i], seriesLength = seriesLength)
        allSizes[i,:] = sizeBirth

    #np.save("../../data/sizeAtBirthPowerSpectrum.npy", allSizes)

In [7]:
allSizes = np.load("../../data/sizeAtBirthPowerSpectrum.npy")
allSizes = allSizes[:,400:] # remove the first 400 points (burn-in)

In [8]:
def powerSpectrum(timeSeries, dt = 1, method = 'rectangle'):
    generations = np.arange(1, len(timeSeries) + 1) # The "time" variable
    N = len(timeSeries) # The number of samples
    #timeSeries = np.hanning(N)*timeSeries
    T = N * dt # The total time of the measurement
    timeSeriesFourier = np.fft.fft(timeSeries - timeSeries.mean()) # The Fourier transform of the signal removing the 0Hz component
    Sxx = 2 * ((dt**2) / T)  * (timeSeriesFourier * np.conj(timeSeriesFourier)) # The power spectrum
    Sxx = Sxx[:int(len(timeSeries) / 2)] # Ignore negative frequencies

    df = 1 / T # The frequency resolution
    fNQ = 1 / (2 * dt) # The Nyquist frequency
    fAxis = np.arange(0, fNQ, df) # The frequency axis

    return fAxis, Sxx.real

In [9]:
app = JupyterDash(__name__)

app.layout = html.Div([
html.Div([

html.Div([
html.H3('Choose the time series'),
dcc.RadioItems(
options = ['trajectory', 'autocorrelation'],
value = 'trajectory',
inline=True,
id = 'timeSeries'
),
], style={'background-color': 'thistle', 'margin': '10px', 'padding': '10px', 'width': '50%', 'border': '1px solid black',  'box-shadow': '5px 5px'}),


html.Div([
html.H3('Choose the method to compute the power spectrum'),
dcc.RadioItems(
options = ['rectangle', 'hanning', 'welch'],
value = 'rectangle',
inline=True,
id = 'method'
),
], style={'background-color': 'aliceblue', 'margin': '10px', 'padding': '10px', 'width': '50%', 'border': '1px solid black',  'box-shadow': '5px 5px'}),

html.Div([
html.H3('Choose the value of gamma'),
dcc.Slider(min=0, max=len(gammaValues) - 1, step = 1, value=0, id = 'gammaValues', updatemode = 'mouseup',\
    marks = {i : {'label': f'{gammaValues[i]:.4f}',  "style": {"transform": "rotate(60deg)"}} for i in range(len(gammaValues))}),
], style = {'background-color': 'lavender', 'margin': '10px', 'padding': '10px',  'border': '1px solid black',  'box-shadow': '5px 5px'}),

html.Div([
html.H3('Choose the scale of the power spectrum'),
dcc.RadioItems(
options = ['linear', 'deciBel'],
value = 'linear',
inline=True,
id = 'unitOfMeasure'
)
], style = {'background-color': 'lightcoral', 'margin': '10px', 'padding': '10px', 'width' : '30%',  'border': '1px solid black',  'box-shadow': '5px 5px'}),


html.Div([
html.H3('Choose the right limit and the bottom limit'),
dcc.Input(
        id='rightXLimit',
        type='number',
        value=0.5, min = 0, max = 0.5
    ),

dcc.Input(
        id='bottomYLimit',
        type='number',
        value=0
    ),
], style = {'background-color': 'lightcyan', 'margin': '10px', 'padding': '10px', 'width' : '30%',  'border': '1px solid black',  'box-shadow': '5px 5px'}),


dcc.Graph(id='powerSpectrum')

])
])
@app.callback(
Output('powerSpectrum', 'figure'),
Input('timeSeries', 'value'),
Input('gammaValues', 'value'),
Input('method', 'value'),
Input('unitOfMeasure', 'value'),
Input('rightXLimit', 'value'),
Input('bottomYLimit', 'value')
)
def update_figure(ts, gammaValue, method, unitOfMeasure, rightXLimit, bottomYLimit):
    if ts == 'trajectory':
        timeSeries = np.exp(allSizes[gammaValue,:])
    else:
        timeSeries = acf(allSizes[gammaValue,:], nlags = 4999, fft = True)
    if method == 'hanning':
        timeSeries = np.hanning(len(timeSeries)) * timeSeries
    if method == 'welch':
        fAxis, Sxx = welch(timeSeries, nperseg = 256)
    else:
        fAxis, Sxx = powerSpectrum(timeSeries = timeSeries, method = method)
    if unitOfMeasure == 'deciBel':
        y = 10 * np.log10(Sxx)
    else:
        y = Sxx
                          
    yTop = np.max(y)
    fig = px.line(x = fAxis, y = y, labels = {'x': 'Frequency', 'y': 'Power'})
    
    fig.update_layout(xaxis_range=[0, rightXLimit], title = 'Power spectrum', yaxis_range = [bottomYLimit, yTop + np.abs(yTop/20)])
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline')

df = pd.DataFrame(np.apply_along_axis(lambda x: powerSpectrum(x)[1], axis = 1, arr = allSizes).T, columns=gammaValues)

Dash is running on http://127.0.0.1:8050/



In [11]:
app = JupyterDash(__name__)

app.layout = html.Div([
html.Div([

html.Div([
html.H3('Choose the time series'),
dcc.RadioItems(
options = ['trajectory', 'autocorrelation'],
value = 'trajectory',
inline=True,
id = 'timeSeries'
),
], style={'background-color': 'thistle', 'margin': '10px', 'padding': '10px', 'width': '50%', 'border': '1px solid black',  'box-shadow': '5px 5px'}),



html.Div([
html.H3('Choose the value of gamma'),
dcc.Slider(min=0, max=len(gammaValues) - 1, step = 1, value=0, id = 'gammaValues', updatemode = 'mouseup',\
    marks = {i : {'label': f'{gammaValues[i]:.4f}',  "style": {"transform": "rotate(60deg)"}} for i in range(len(gammaValues))}),
], style = {'background-color': 'lavender', 'margin': '10px', 'padding': '10px',  'border': '1px solid black',  'box-shadow': '5px 5px'}),






dcc.Graph(id='powerSpectrum')

])
])
@app.callback(
Output('powerSpectrum', 'figure'),
Input('timeSeries', 'value'),
Input('gammaValues', 'value'),
)
def update_figure(ts, gammaValue):
        Fs = 1
        overlap = 0.95 * Fs
        if ts == 'trajectory':
                timeSeries = np.exp(allSizes[gammaValue,:])
        else:
                timeSeries = acf(allSizes[gammaValue,:], nlags = 4999, fft = True)
        f, t, Sxx = spectrogram(
        timeSeries,                  # Provide the signal,
        fs=Fs,                # ... the sampling frequency,
        nperseg=500,     # ... the length of a segment,
        noverlap=overlap) 
        
                                
        
        

        fig = go.Figure(data=go.Heatmap(
        z=Sxx,
        x=t,
        y=f,
        colorscale='jet'))
        fig.update_layout(
        width = 700,
        height = 500,
        )
        fig.update_layout(
        title='Spectrogram',
        xaxis_title='Generation',
        yaxis_title='Frequency',
        )
        #fig.update_layout(xaxis_range=[0, rightXLimit], title = 'Spectrogram')#, yaxis_range = [bottomYLimit, yTop + np.abs(yTop/20)])
        return fig

if __name__ == '__main__':
    app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

